In [1]:
import os, re, csv
import pandas as pd
import numpy as np

In [36]:
eta_df = pd.read_csv("../자료/질문2.csv", encoding='ANSI')
eta_df.head(10)

eta_df = eta_df.replace(np.nan, "['']", regex=True)

eta_df.head(10)

,Unnamed: 0,내용,댓글,댓글쓴이,대댓글여부,id
0,0,안녕하세요 조교님! 혹시 부전공 신청은 막학기에 불가능한가요??ㅜ이번학기가 막학기고...,[''],[''],[''],112883775
1,1,"안녕하세요, 조교님!저는 전역이 5주 정도 남은 군인입니다. 전역 후 계획을 세우다...",[''],[],[''],112880808
2,2,방학중 어학연수에 대해 질문이 있어서요!휴학중 학점인정은 재학 기간중 1회에 한해서...,[''],[],[''],112864005
3,3,안녕하세요 조교님!제가지금 4학년 1학기 인데4학년은 1학기에 12학점 미만으로 학...,[''],[],[''],112848838
4,4,조교님 안녕하세요!재수강을 하려고 하는데재수강과목선택란에 조회결과가 없다고 뜨면다시...,[''],[],[''],112813045
5,5,안녕하세요 조교님!!자퇴를 하려고 하는데 어디서 신청하면 되나요??,"['종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 ...",['에타조교'],['parent'],112748915
6,6,안녕하세요 조교님!1/4학기 이전에 휴학 신청을 하면 등록금 환불이라고 들었는데 맞...,['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 ...,"['에타조교', '익명(글쓴이)', '에타조교', '익명(글쓴이)']","['parent', 'child', 'child', 'child']",112663087
7,7,안녕하세요 조교님!!ㅠㅠ제가 항상 동일과목을 재수강 해서 자동연결이 됐어서이번에도 ...,"['음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면...","['에타조교', '익명(글쓴이)', '에타조교', '익명(글쓴이)']","['parent', 'child', 'child', 'child']",112627947
8,8,"안녕하세요 조교님!이번에 인성을 위한 lectio(언어와 문학, p/f 과목)을 듣...",['미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동...,['에타조교'],['parent'],112619096
9,9,안녕하세요 조교님! 제가 이번학기에 수강하는 언어와 문학 과목을 재수강 연결 하려고...,['대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (...,['에타조교'],['parent'],112563109


In [13]:
content = eta_df['내용'].tolist()
comments = eta_df['댓글'].tolist()
writer = eta_df['댓글쓴이'].tolist()
ids = eta_df['id'].tolist()

In [14]:
comments[:10]

["['']",
 "['']",
 "['']",
 "['']",
 "['']",
 "['종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종지에 제출하면 됩니다']",
 "['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다', '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다', '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..', '아 넵 알겠습니다']",
 "['음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학기 상관없는 동일수업인지 알 수가 없어서 답변이 제한되네요...우선 종정시에 접속하셔서 수업/수강관리 > 재수강과목선택 부분을 확인바랍니다', '전공수업에 1,2 나눠지지 않은 동일과목입니다! 자동연결될줄 알았는데 학교에서 미연결과목있다고 문자오고 재수강과목선택 들어가서 성적연결신청을 누르면 이전성적 조회결과가 없다고 뜹니다ㅠㅠ 교수님까지 똑같은 강의인데ㅠㅠ 자동연결이 되는걸까요ㅠㅠ', '그럼 자동연결이 되었을텐데 이상하네요 혹시 전공 말고 교양 등 다른 과목이 아닌지 확인바라며 만일 그것도 아니라면 학종지에 문의바랍니다', '원래 재수강 연결할때 이전성적이 뜨고 연결할수있는건가요? 전 아무리봐도 같은 과목인데 자동연결이라고 안써져있고..ㅠㅠ']",
 "['미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']",
 "['대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']"]

In [27]:
#리스트로 변환 방식 예시
a = "['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다', '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다', '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..', '아 넵 알겠습니다']"
a =re.sub("^\[[\"\']", '', a)#양 끝의 "[", "]" 제거
a =re.sub("[\"\']\]$", '', a)
test =re.split("[\"\'], [\"\']", a) # ', "   ", '  ', ' 등을 기준으로 split하여 리스트로 만든다. 
test

['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다',
 '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다',
 '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..',
 '아 넵 알겠습니다']

In [8]:
def string_to_list(str_ls):
    res = []
    for s in str_ls:
        temp = re.sub("^\[[\"\']", '', s)
        temp = re.sub("[\"\']\]$", '', temp)
        res.append(re.split("[\"\'], [\"\']", temp))
    return res

In [38]:
comments_ls = string_to_list(comments)
writer_ls = string_to_list(writer)
print(comments_ls[:10])
# print(writer_ls[:10])

[[''], [''], [''], [''], [''], ['종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종지에 제출하면 됩니다'], ['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다', '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다', '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..', '아 넵 알겠습니다'], ['음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학기 상관없는 동일수업인지 알 수가 없어서 답변이 제한되네요...우선 종정시에 접속하셔서 수업/수강관리 > 재수강과목선택 부분을 확인바랍니다', '전공수업에 1,2 나눠지지 않은 동일과목입니다! 자동연결될줄 알았는데 학교에서 미연결과목있다고 문자오고 재수강과목선택 들어가서 성적연결신청을 누르면 이전성적 조회결과가 없다고 뜹니다ㅠㅠ 교수님까지 똑같은 강의인데ㅠㅠ 자동연결이 되는걸까요ㅠㅠ', '그럼 자동연결이 되었을텐데 이상하네요 혹시 전공 말고 교양 등 다른 과목이 아닌지 확인바라며 만일 그것도 아니라면 학종지에 문의바랍니다', '원래 재수강 연결할때 이전성적이 뜨고 연결할수있는건가요? 전 아무리봐도 같은 과목인데 자동연결이라고 안써져있고..ㅠㅠ'], ['미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)'], ['대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']]


In [41]:
#에타 조교의 첫번째 답변만 추출, 댓글쓴이와 댓글은 그 개수가 1대1 대응
answer = []
found = 0 #에타조교를 찾았는지 여부
for i, c in enumerate(comments_ls):
    found = 0 #찾았음 표시 토글 초기화
    for j, w in enumerate(writer_ls[i]):
        if w=="에타조교": #에타조교가 가장 처음으로 쓴 댓글을 만나면, 
            answer.append(c[j]) #현재 comment들 중 에타조교가 쓴 글의 인덱스를 answer에 추가
            found = 1 #찾았음을 표시
            break
    #에타조교가 쓴 글이 없으면,
    if found == 0:
        answer.append("no answer by 에타조교")
print(len(answer), len(ids)) #개수가 일치하는 지 확인
answer[:10]


3024 3024


['no answer by 에타조교',
 'no answer by 에타조교',
 'no answer by 에타조교',
 'no answer by 에타조교',
 'no answer by 에타조교',
 '종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종지에 제출하면 됩니다',
 '1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다',
 '음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학기 상관없는 동일수업인지 알 수가 없어서 답변이 제한되네요...우선 종정시에 접속하셔서 수업/수강관리 > 재수강과목선택 부분을 확인바랍니다',
 '미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)',
 '대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']

In [46]:
raw_data = {'질문': content, 
            '답변': answer, 
            'id': ids}
data = pd.DataFrame(raw_data)
data[:10]

,질문,답변,id
0,안녕하세요 조교님! 혹시 부전공 신청은 막학기에 불가능한가요??ㅜ이번학기가 막학기고...,no answer by 에타조교,112883775
1,"안녕하세요, 조교님!저는 전역이 5주 정도 남은 군인입니다. 전역 후 계획을 세우다...",no answer by 에타조교,112880808
2,방학중 어학연수에 대해 질문이 있어서요!휴학중 학점인정은 재학 기간중 1회에 한해서...,no answer by 에타조교,112864005
3,안녕하세요 조교님!제가지금 4학년 1학기 인데4학년은 1학기에 12학점 미만으로 학...,no answer by 에타조교,112848838
4,조교님 안녕하세요!재수강을 하려고 하는데재수강과목선택란에 조회결과가 없다고 뜨면다시...,no answer by 에타조교,112813045
5,안녕하세요 조교님!!자퇴를 하려고 하는데 어디서 신청하면 되나요??,"종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종...",112748915
6,안녕하세요 조교님!1/4학기 이전에 휴학 신청을 하면 등록금 환불이라고 들었는데 맞...,1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2....,112663087
7,안녕하세요 조교님!!ㅠㅠ제가 항상 동일과목을 재수강 해서 자동연결이 됐어서이번에도 ...,"음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학...",112627947
8,"안녕하세요 조교님!이번에 인성을 위한 lectio(언어와 문학, p/f 과목)을 듣...",미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과...,112619096
9,안녕하세요 조교님! 제가 이번학기에 수강하는 언어와 문학 과목을 재수강 연결 하려고...,대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일...,112563109


In [49]:
data.to_csv("../자료/에타조교_정답추출.csv", mode='w',index=False, encoding='utf-8')

In [14]:
QA_df = pd.read_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\results\Q&A.csv", encoding='ANSI')
QA_df = QA_df.fillna("['']")
QA_df

,QUESTION,ANSWER,TAGSET,LEMMA
0,이중전공이랑 부전공 차이에 대해서 궁금해요이중전공은 학위졸업장 표시인건 알겠는데부전...,[''],"['DM', 'SM']",이중전공 부전공 차이 대하 이중전공 학위 졸업장 표시 부전공 졸업장 표시
1,재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴를 들어갔는데안내사...,[''],['UK'],재수강 연결 종합 정보 시스템 수업 수강 관리 수강 과목 선택 메뉴 들어가 안내 사...
2,항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라 막학기에 수업을 더 ...,[''],['UK'],수고 다르 아니 제가 평점 학기 수업 평점 올리 졸업 학점 134 학점 초과 문제
3,제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 변경하려고 시도했는...,[''],"['SM', 'DM']",처음 이중전공 신청 이중전공 경영 변경 시도 자체 시험 떨어지 변경 실패 지나 겨울...
4,이중전공을 바꾸고싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요 아니...,[''],['DM'],이중전공 바꾸 바꾸 학점 소멸
...,...,...,...,...
7010,광역특화전공이 뭔가요,국가주도 CK-Ⅱ 사업의 일환으로 2015학년도부터 신설되는 ‘광역특화’ 융합전 공...,['UK'],광역 특화 전공
7011,디지털인문학전공이 뭔가요,한국의 인문학적 전통과 문화에 대한 심층적 지식을 바탕으로 디지털 매체를 활용한 해...,['UK'],디지털 인문학 전공
7012,디지털인문학전공의 연계기초과목엔 뭐가 있나요,"한국철학사(철학과), 한국사입문(사학과) 또는 한국문화사(사학 부전공), KFL문법...",['UK'],디지털 문학 전공 연계 기초 과목
7013,디지털인문학전공 연계기초 과목 학점인정은 어떻게 하나요,수강신청 후 학사종합지원센터 단과대 교학파트(본인 1전공 소속 기준)에 방문하여 이...,['UK'],디지털 인문학 전공 연계 기초 과목 학점 인정 어떻


In [34]:
qa_content = QA_df['QUESTION'].tolist()
qa_comments = QA_df['ANSWER'].tolist()

qa_comments_ls = string_to_list(qa_comments)

print(qa_comments_ls[3000:3100])

[[''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], ['종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종지에 제출하면 됩니다'], ['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다', '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다', '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..', '아 넵 알겠습니다'], ['음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학기 상관없는 동일수업인지 알 수가 없어서 답변이 제한되네요...우선 종정시에 접속하셔서 수업/수강관리 > 재수강과목선택 부분을 확인바랍니다', '전공수업에 1,2 나눠지지 않은 동일과목입니다! 자동연결될줄 알았는데 학교에서 미연결과목있다고 문자오고 재수강과목선택 들어가서 성적연결신청을 누르면 이전성적 조회결과가 없다고 뜹니다ㅠㅠ 교수님까지 똑같은 강의인데ㅠㅠ 자동연결이 되는걸까요ㅠㅠ', '그럼 자동연결이 되었을텐데 이상하네요 혹시 전공 말고 교양 등 다른 과목이 아닌지 확인바라며 만일 그것도 아니라면 학종지에 문의바랍니다', '원래 재수강 연결할때 이전성적이 뜨고 연결할수있는건가요? 전 아무리봐도 같은 과목인데 자동연결이라고 안써져있고..ㅠㅠ'], ['미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)'], ['대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)'], ['조교를 거치지 않고 교수님 개인 메일로 보내는 거라면 안심하셔도 됩니다 다른 학생들에게 노출

In [56]:
answer = []
print(len(qa_comments_ls))
for x in qa_comments_ls:
    if x == ['']:
        answer.append("No Answer")
    else:
        for ans in x:
            if len(ans) == 1:
                answer.append(ans)
            else:
                print(len(ans))
print(answer[3000:3030])
print(len(answer))


81
56
79
25
76
32
32
226
74
5
62
92
100
29
15
6
72
10
29
13
41
60
89
19
84
63
166
45
5
103
9
56
38
19
27
2
10
61
88
96
41
80
105
5
39
14
190
36
77
9
86
12
19
50
162
109
95
48
159
142
12
22
6
55
219
75
73
131
17
108
130
16
90
43
15
49
37
6
30
8
23
8
152
10
49
62
190
237
53
7
118
56
12
27
35
27
2
59
5
85
40
11
6
9
69
42
27
32
155
97
6
67
51
7
30
49
127
24
58
42
26
40
40
49
71
64
21
33
53
125
34
6
51
27
7
174
135
25
72
94
6
75
55
13
7
92
66
71
35
161
46
94
115
41
10
50
56
132
7
67
7
55
9
19
189
77
32
54
214
26
45
83
40
67
11
31
38
158
273
12
131
44
153
13
31
124
107
189
87
11
53
7
98
51
124
86
38
17
11
65
114
11
66
178
81
36
6
192
89
7
64
82
49
21
93
30
24
13
10
96
92
61
26
7
124
159
84
33
46
8
87
9
9
113
160
35
247
101
8
101
55
86
12
255
52
6
211
56
7
137
66
23
41
39
76
93
88
35
46
47
80
128
43
116
76
63
8
45
189
16
76
63
33
38
86
7
47
29
6
53
58
243
52
148
81
225
11
43
178
144
85
25
69
245
54
11
289
162
59
42
7
110
41
28
6
299
186
157
10
110
46
13
19
9
212
77
105
8
15
157
20
153
88
16
